# Professor Evaluation to Find Corresponding Student

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from utils import *

%load_ext autoreload
%autoreload 2

In [ ]:
# Define a function to extract subtables based on a condition (e.g., a specific marker or blank rows)
def extract_subtables(df):
    subtables = []
    subtable = []
    for _, row in df.iterrows():
        if row.isnull().all():
            if subtable:
                subtables.append(pd.DataFrame(subtable))
                subtable = []
        else:
            subtable.append(row)
    if subtable:
        subtables.append(pd.DataFrame(subtable))
    return subtables
def extract_student_application_ID(df):
    print("read student application ID")
    df.columns = df.iloc[0]
    df = df[1:]
    name_app_id_dict = df.set_index('Name')['Application ID'].to_dict()
    return name_app_id_dict

# extract student information from each round 
def extract_tables(roundNum):
    file_path = f'YOUR_STUDENT_DATA.xlsx' #replace with real data
    xls = pd.ExcelFile(file_path)

    # Display the sheet names
    print(xls.sheet_names)

    # Read each sheet into a DataFrame
    data = {}
    for sheet_name in xls.sheet_names:
        data[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name)
    # Extract subtables from each sheet
    subtables_dict = {}
    for sheet_name, df in data.items():
        subtables_dict[sheet_name] = df

    # Display subtables
    df = []

    for sheet_name, subtables in subtables_dict.items():
        print(f"\nSheet: {sheet_name}")
        # if "result" in sheet_name:
        #     temp_dict = extract_student_application_ID(subtables)
        # elif "detail" in sheet_name:

        if "result" in sheet_name:
            temp_dict = extract_student_application_ID(subtables)
            student_ID_list.update(temp_dict)


        if "original" in sheet_name or "details" in sheet_name:
            subDF = subtables
            primary_columns = subDF.columns
            secondary_columns = subDF.iloc[0]

            # Combine them, using the secondary name if the primary is None
            columns = [
                primary if "Unnamed" not in str(primary) else secondary
                for primary, secondary in zip(primary_columns, secondary_columns)
            ]

            # Set the new columns to the DataFrame and drop the first two rows
            subDF.columns = columns

            subDF = subDF.drop([0])

            # Reset index
            subDF = subDF.reset_index(drop=True)
            subDF = subDF.dropna(subset=['最终得分'])
            subDF = subDF.ffill()
            subDF = subDF.reset_index(drop=True)
            # print(subDF.columns)
            # Initialize lists to store the structured data
            mapping = {
                '学生': 'name',
                '申请号': 'id',
                '邮箱': None,  # Not needed in the provided mapping
                '主席决策': None,  # Not needed in the provided mapping
                '委员会推荐': None,
                '分委会推荐': None,
                '教授评分': "recommend score",  # in r7,8
                # '教授评分': None, # in r10
                '教授推荐汇总': None,  # Not needed in the provided mapping
                '分数': "score",  # Not needed in the provided mapping
                '教授': "professor",  # Not needed in the provided mapping
                '科目': 'major',
                'GPA': 'gpa',
                '数学能力': 'math ability',
                '语言文字综合能力': 'mastery of language',
                '主题核心课程成绩': 'specialty core courses',
                '实验课成绩': 'lab courses',
                '毕业设计/作品及成绩': 'capstone/design project',
                '企业/临床实习成绩': 'internship',
                '社会服务活动': 'community service',
                '学生社团活动': 'student clubs',
                '国内国际大赛成绩': 'major int\'l/national competition',
                '本科毕业院校': 'college of bachelor degree',
                '最终得分': "overall score"  # Not needed in the provided mapping
            }
            # filtered_mapping = {k: v for k, v in mapping.items() if v is not None}
            subDF.rename(columns=mapping, inplace=True)
            subDF = subDF.drop(columns=[None])
            df.append(subDF)
            print("Length of sub table: "+str(len(subDF)))

    #merge sub dataframe into one dataframe        
    df=pd.concat(df)
    return df

In [ ]:
student_ID_list={}
table7=extract_tables(7)
table8=extract_tables(8)
table10=extract_tables(10)

#unify the id in each table
table7['id'] = table7['name'].map(student_ID_list)
for id in table8['id']:
    newID = id[:11]
    table8.loc[table8['id'] == id, 'id'] = newID
for id in table10['id']:
    newID = int(id)
    table10.loc[table10['id'] == id, 'id'] = newID    
    
df=pd.concat([table7, table8, table10])
df=df.reset_index(drop=True)
df['id']=pd.to_numeric(df['id'])

In [61]:
#match name with id if id is not in the sheet
# df['id'] = df['id'].fillna(df['name'].map(student_ID_list))
# df['id']=pd.to_numeric(df['id'])

In [102]:
#map the recommend score
recommmend_score_mapping = {
    '强烈推荐': 2,
    '推荐': 1,
    '居中': 0,
    '不推荐': -1
}

df['recommend score']=df['recommend score'].map(recommmend_score_mapping)
df['score']=df.groupby('id')['recommend score'].transform('sum')
df.drop('recommend score', axis=1, inplace=True)

# Calculate Mean and STD of Professor's Evaluation

In [ ]:
numerical_cols = df.select_dtypes(include='number').columns
results = pd.DataFrame()

def mean_ignore_zeros(x):
    return x[x != 0].mean()

# Custom function to calculate std ignoring zeros
def std_ignore_zeros(x):
    # print(x[x == 0].sum())
    return x[x != 0].std()

for col in numerical_cols:
    grouped = df.groupby('id')[col].agg(mean_ignore_zeros).rename(f'{col}_mean')
    grouped_std = df.groupby('id')[col].agg(std_ignore_zeros).rename(f'{col}_std')
    
    if results.empty:
        results = grouped.to_frame().join(grouped_std)
    else:
        results = results.join(grouped.to_frame()).join(grouped_std)
results = results.reset_index()

std_columns = [i for i in results.columns if "std" in i]
mean_columns = [i for i in results.columns if "mean" in i]
print(results[mean_columns].mean())
results[std_columns] = results[std_columns]**2
sqrt_mean_square = np.sqrt(results[std_columns].mean())
print(sqrt_mean_square)

# Read Student Profile

In [108]:
not_find_list = []
find_list=[]
def read_files_with_id_in_folder(folder_path, ID):
    # Iterate through all files in the specified folder
    profile = None
    for filename in os.listdir(folder_path):
        # Construct absolute path to file
        file_path = os.path.join(folder_path, filename)
        
        # Check if the current path is a file and not a directory
        if os.path.isfile(file_path) and ".pdf" in file_path and str(ID) in file_path:
            print(f"Found file: {filename}")
            profile = extract_student_profile(file_path)
            profile["Proposed Research Plan / Vision Statement"] = profile["Proposed Research Plan / Vision Statement"][:10000]
            profile["/ CV"] = profile["/ CV"][:4000]
            if len(json.dumps(profile)) > 20000:
                print(f"warning: {filename}, string len: {len(json.dumps(profile))}")
            # profile
            # If you need to read the file, you can open it here
            # with open(file_path, 'r') as file:
            #     content = file.read()
            #     # Do something with the content
            #     print(content)
    if not profile:
        print(f"Not find {ID}")
        not_find_list.append(ID)
    return profile

def check_files_existence_with_id(folder_path, ID):
    # Iterate through all files in the specified folder
    check=False
    for filename in os.listdir(folder_path):
        # Construct absolute path to file
        file_path = os.path.join(folder_path, filename)

        # Check if the current path is a file and not a directory
        if os.path.isfile(file_path) and ".pdf" in file_path and str(ID) in file_path:
            check=True
            find_list.append(ID)
    if not check:
        print(f"Not find {ID}")
        not_find_list.append(ID)
    return find_list 


In [ ]:
path = "STUDENT_PROFILE_PATH"
def rename_files(folder_path):
    for filename in os.listdir(folder_path):
        # Check if the filename contains a dot
        new_name = filename.replace('.pdf', '').replace('pdf', '')
        print(new_name)
        if '.' in new_name:
            # Get the new filename
            new_name = "".join(filename.split(".")[1])
        if '_' in new_name:
            new_name = filename.split("_")[0]
        new_name = new_name[:11] + ".pdf"
        # Full old and new file paths
        old_file = os.path.join(folder_path, filename)
        new_file = os.path.join(folder_path, new_name)
        # Rename the file
        try:
            os.rename(old_file, new_file)
        except:
            print(new_file + "   exist")
        print(f'Renamed: {filename} to {new_name}')

# Call the function
# rename_files(path)

# Read File From Raw PDF File

In [ ]:
path = "STUDENT_PROFILE_PATH"
read_file_list = []

In [ ]:
for i, row in results.iterrows():
    row_id = int(row['id'])
    profile = read_files_with_id_in_folder(path, row_id)
    read_file_list.append(profile)


In [ ]:
#convert read list to json

student_info_path = "Processed_STUDENT_PROFILE_PATH"
important_keys = ["Education Background", "Awards and Professional Qualifications",
                  "Publications", "English Language Proficiency", "Proposed Research Plan / Vision Statement",
                  "/ CV", "Extracurricular Activities / Volunteer Work", "Taken Courses"]
for profile in tqdm(read_file_list):
    ID = profile["ID"]
    filtered_profile = {key: profile[key] for key in important_keys}
    if not os.path.exists(student_info_path):
        # Create a new directory because it does not exist
        os.makedirs(student_info_path)
    with open(f'{student_info_path}\\{ID["identity"][1]}.json', 'w') as fp:
        json.dump(profile, fp)

In [ ]:
# Get all missing student id/name

path = "STUDENT_PROFILE_PATH"
read_file_list = []
for i, row in results.iterrows():
    row_id = int(row['id'])
    exist_file_list = check_files_existence_with_id(path, row_id)
    #read_file_list.append(profile)

str_not_find_list=[str(i) for i in not_find_list]
missing_student_file=df.loc[df['id'].isin(str_not_find_list)]
missing_student_file.drop_duplicates(subset=['id'],inplace=True)
missing_student_file=missing_student_file.loc[:,['name','id']]
missing_student_file

# Read File From Existing Result

In [ ]:
path = "STUDENT_PROFILE_PATH"
read_path = "Processed_STUDENT_PROFILE_PATH"

In [ ]:
# missing_num=0
# for filename in os.listdir(read_path):
#     file_path = os.path.join(read_path, filename)
#     if os.path.isfile(file_path) and ".json" in file_path:
#         with open(file_path, 'r') as file:
#             data = json.load(file)
#         if len(data["Awards and Professional Qualifications"])==0:
#             missing_num+=1
#             print(data["ID"])
# print(missing_num)

In [3]:
read_file_list = []

for filename in os.listdir(read_path):
    file_path = os.path.join(read_path, filename)
    if os.path.isfile(file_path) and ".json" in file_path:
        with open(file_path, 'r') as file:
            data = json.load(file)
        if 'response' in data:
            del data['response']
        read_file_list.append(data)

In [ ]:

#enter you gpt related information here
url = ""

headers = {
    "Content-Type": "application/json",
    "Authorization": ""
}

new_data = []

def request_test():
    data = {
        "model": "gpt-4",
        "messages": [{"role": "system", "content": "you are a test"},
                     {"role": "user", "content": "Write a test"}],
        # "max_tokens": 4096,
        "temperature": 0.0
    }
    response = requests.post(url, headers=headers, data=json.dumps(data)).json()
    time.sleep(3)
    return response


def request(system_prompt, profile):
    data = {
        "model": "gpt-4",
        "messages": [{"role": "system", "content": system_prompt},
                    {"role": "user", "content": json.dumps(profile)}],
        # "max_tokens": 4096,
        "temperature": 0.0
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(data)).json()
        #print(len(data["messages"][0]["content"])+len(data["messages"][1]["content"]))
        response = response['choices'][0]['message']['content']
        time.sleep(3)
        return response
    except Exception as e:
        print(e)
        time.sleep(3)



system_prompt = ("You are a professor and interviewer who reviews the application of Master of Philosophy in Top-tier university, \
                and you are expected to analyze the applicants background in critical ways, be aware that you should use your background information on evaluating the school, award, publication, and company reputation. \
                Overall, you should analyze the background in following steps: \
                1. Categorized them into one of the following majors: Science, Engineering, Liberal Arts, Arts, Design, Business, Management, Medical \
                2. Rate the Student into the following criteria from 1 to 100, including: GPA, Math Ability, Mastery of Language, Specialty core courses, Lab courses, Capstone/Design project \
                , Internship, Community service, Student clubs, Major Int'l/National Competition, College of Bachelor Degree, College of higher degree. The rate range from 60 to 70 is very poor. 70 to 80 is normal. 80 to 90 is great. Above 90 is very outstanding. For most of the time, you should not give above 95 or below 60 unless the student did very well or very poor.\
                In GPA, normally 3.3/4.0, which is equivalent to 4.125/5.0, are consider fair gpa, which should be rated above 80. GPA below 3.0/4.0, which is equivalent to 3.75/5.0, should be below 75. Higher rank school, based on QS World University Rankings instead of reputation, can accept relative low GPA, and vice versa. You should focus more of the student's GPA during Bachelor's degree instead of the GPA during Master's degree. \
                In Mastery of Language, the full score of IELTS is 9.0, full score of TOEFL is 120, and full score of Duolingo is 160. 6.5/9.0 in IELTS, 90/120 in TOEFL, and 110/160 in Duolingo is consider a fair grade and you can give above 80. You should rate based on the overall score instead of the subject score in TOEFL, IELTS, or Duolingo. \
                In Math Ability, Specialty core Courses, and Lab courses, you should not only consider what course the student completed but also what grade the student got in that course. You need to read the student's grade from the \"Taken Courses\" based on the chart's format, like course, credit, grade. If the student have a low grade, below B- or 80/100, in multiple courses, you should give a low grade. Higher rank school, based on QS World University Rankings instead of reputation, can accept relative low grade, and vice versa.\
                In Capstone/Design project, you should be critical on Proposed Research Plan / Vision Statement, and the project the students did, most student writing seemingly meaningful research, while as professor you should verify if it is the real research problem. Unless the project is significant impactful, insightful or novel, you should not give score higher than 85. For a normal project, it should be around 75. You should not rate the project based on the score the student tell you. \
                In Award, be aware that the Mathematical Contest in Modeling is worth nothing only if it is the first price. For competition, National award could consider as high score. Otherwise, you should be critical in evaluating if the award is good or not.\
                ONLY in \"Community service\", you could give 70 if the student did not provide any information. For any other categories that the student did not provide information, you need to give NA/100. \
                In College of Bachelor Degree and College of higher degree, you need to compare the student's college's rank, categories it into one of the tier, and rate according to the score in the parenthesis.\
                Tier 1 college(above 90 points): Sun Yat-Sen University, Tianjin University, etc. Tier 2 college(80-90 points): South China University of Technology, Nanchang University, etc. Tier 3 college(below 80 points): Hainan University, Hangzhou Normal University, etc.  \
                The mean of the rating is 75, and the standard deviation is 11, you should differentiate the student to make sure the pass or fail is clear.\
                3. Write a short summary about the evaluation on student\n\n \
                4. Calculate the Overall Score for student based on previous score you give to that student. You should take the completeness of the student information into account, missing too many essential information will lead to score deduction in overall score. You must give an overall score. \
                5. Determine if the student could pass or not, the pass rate is 65%.\
                You should strictly follow the template rule for the response: \
                1. The score must be in front of the description e.g. GPA: 70/100 (original gpa 3.0/4.0)\
                2. Even you infer student ability you should include the inferring process in () e.g. Math Ability: 70/100 (Given the range of engineering and energy-related courses)\
                The following are the response example: \
                1. Major: Engineering (Industrial Engineering) \
                2. Rating: \
                  - GPA: 85/100 (Final GPA of 3.72/4.0 from New Jersey Institute of Technology)\
                  - Mastery of Language: 81/100 (IELTS score of 7.0)\
                  - Math Ability: 81/100 (Complete courses like Advanced Engineering Statistic with score of 80) \
                  - Specialty core courses: 88/100 (Courses like Advanced Engineering Statistics with score of A- indicate great specialization)\
                  - Lab courses: 75/100 (Participation in lab courses like Fundamentals Physics Experiment with score above 70) \
                  - Capstone/Design project: 75/100 (Research projects and publications) \
                  - Internship: 70/100 (Internship experiences at Tencent Interactive Entertainment) \
                  - Community service: 80/100 (Participation in University Art Troupe) \
                  - Student clubs: 70/100 (No information provided) \
                  - Major Int'l/National Competition: 80/100 (Third place in Mock Trading Contest) \
                  - College of Bachelor Degree: 84/100 (New Jersey Institute of Technology) \
                  - College of higher degree: 75/100 (University of Southampton) \
                3. Summary: \
                  XXXXXX \
                4. Overall Score: 78 \
                5. Passed or Unpassed: Unpassed")

In [ ]:
request_test() 

In [6]:
important_keys = ["Education Background", "Awards and Professional Qualifications",
                  "Publications", "English Language Proficiency", "Proposed Research Plan / Vision Statement",
                  "/ CV", "Extracurricular Activities / Volunteer Work", "Taken Courses"]
save_path = f"{path}\\Result_v12"

In [ ]:

for profile in tqdm(read_file_list):
    ID = profile["ID"]
    filtered_profile = {key: profile[key] for key in important_keys}
    response = request(system_prompt,filtered_profile)
    profile["response"] = response
    if not os.path.exists(save_path):
        # Create a new directory because it does not exist
        os.makedirs(save_path)
    with open(f'{save_path}\\{ID["identity"][1]}.json', 'w') as fp:
        json.dump(profile, fp)

In [144]:
def rerunEmptyResponse(fileID):
    profile=None
    file_path = os.path.join(read_path, f"{fileID}.json")
    if os.path.isfile(file_path) and ".json" in file_path:
        with open(file_path, 'r') as file:
            data = json.load(file)
        if 'response' in data:
            del data['response']
        profile=data
    
    if profile is not None:
        ID = profile["ID"]
        filtered_profile = {key: profile[key] for key in important_keys}
        print(filtered_profile)
        response = request(system_prompt,filtered_profile)
        profile["response"] = response
        if not os.path.exists(save_path):
            # Create a new directory because it does not exist
            os.makedirs(save_path)
        with open(f'{save_path}\\{ID["identity"][1]}.json', 'w') as fp:
            json.dump(profile, fp)

In [145]:
import pandas as pd

def decompose_response(response):
    response = response.replace("**", "").replace("-", "")
    sections = response.strip().split('\n\n')

    # Initialize lists to store data
    major = None
    ratings = {}
    summary = None

    
    colon_type = ":"
    # Loop through each line and extract data
    for section in sections:
        lines = section.strip().split('\n')
        if 'Major' in lines[0]:
            major = lines[0].split(colon_type)[1].strip().split('(')[0]
        if 'Rating' in lines[0]:
            for line in lines[1:]:
                category, score_and_des = line.split(colon_type)[0], "".join(line.split(colon_type)[1:])
                category = category.strip().lower()
                score = score_and_des.split('(')[0].strip().split('/')[0][-2:]
                try:
                    description = score_and_des.split('(')[1].split(')')[0]
                except:
                    description = ""
                try:
                    score = float(score)
                except ValueError:
                    score = -1
                ratings[category] = score
                ratings[category + "_description"] = description
        elif 'Summary' in lines[0]:
            summary = ' '.join(lines[1:]).strip()
        elif 'Overall Score' in lines[0]:
            line = lines[0]
            category, score_and_des = line.split(colon_type)[0], "".join(line.split(colon_type)[1:])
            score = score_and_des.split('(')[0].strip().split('/')[0].strip()
            try:
                description = score_and_des.split('(')[1].split(')')[0]
            except:
                description = ""
            try:
                score = float(score)
            except ValueError:
                score = -1
            ratings["llm pred score"] = score
        elif 'Passed or Unpassed' in lines[0]:
            line = lines[0]
            ratings["llm pred pass"] = line.split(colon_type)[1].split('(')[0].strip() == "Passed"

    # Create a dataframe
    df = pd.DataFrame(ratings, index=[0])

    # Add Major column
    df['Major'] = major
    df["Summary"] = summary
    return df

In [146]:
import re
import pandas as pd

def decompose_response_re(response):
    # Initialize lists to store data
    major = None
    ratings = {}
    summary = None

    # Define regular expressions
    major_regex = re.compile(r'Major:\s*(.*?)(?:\(|$)')
    rating_regex = re.compile(r'\*\*\s*(.*?)(?::|:\*\*):\s*(\d+/\d+|\d+|N/A)\s*(?:\((.*?)\))?')
    summary_regex = re.compile(r'Summary:\n([\s\S]*)')

    # Extract major
    major_match = major_regex.search(response)
    if major_match:
        major = major_match.group(1).strip()

    # Extract ratings
    rating_section = re.search(r'Rating:\s*([\s\S]+?)\n\n', response)
    if rating_section:
        for match in rating_regex.finditer(rating_section.group(1)):
            category = match.group(1).strip()
            score = match.group(2).strip().split('/')[0]
            description = match.group(3) if match.group(3) else ""
            try:
                score = float(score)
            except ValueError:
                score = -1
            ratings[category] = score
            ratings[category + "_description"] = description

    # Extract summary
    summary_match = summary_regex.search(response)
    if summary_match:
        summary = summary_match.group(1).strip()

    # Create a dataframe
    df = pd.DataFrame(ratings, index=[0])

    # Add Major and Summary columns
    df['Major'] = major
    df["Summary"] = summary
    
    return df

In [ ]:
path = save_path

read_file_list = []
df_list = []
save_path = f"{path}"
save_csv_path = f"{path}\\csv_summary"
for filename in os.listdir(save_path):
    # Construct absolute path to file
    file_path = os.path.join(save_path, filename)
    
    # Check if the current path is a file and not a directory
    if os.path.isfile(file_path) and ".json" in file_path:
        # print(f"Found file: {filename}")
        # if "62350081679" in file_path:
        with open(file_path, 'r') as fp:
            data = json.load(fp)
        #print(data["response"])
        
        #Skip when response is NULL
        if(data["response"]==None):
            print(data["ID"]["identity"][1]+"'s response is None")
            rerunEmptyResponse(data["ID"]["identity"][1])
            with open(file_path, 'r') as fp:
                data = json.load(fp)
                
        decomposed_text = decompose_response(data["response"])
        decomposed_text["id"] = data["ID"]["identity"][1]
        decomposed_text["name"] = data["ID"]["identity"][0]
        # cols = decomposed_text.columns
        # cols.insert(0, cols.pop(cols.index('id')))
        # cols.insert(1, cols.pop(cols.index('name')))
        df_list.append(decomposed_text)
        # read_file_list.append(extract_student_profile(file_path))
        # If you need to read the file, you can open it here
        # with open(file_path, 'r') as file:
        #     content = file.read()
        #     # Do something with the content
        #     print(content)

decomposed_df = pd.concat(df_list, ignore_index=True)
cols = decomposed_df.columns.tolist()
id_index=cols.index("id")
cols = cols[id_index:] +cols[id_index-2:id_index] + cols[:id_index-2]
decomposed_df = decomposed_df[cols]
print(decomposed_df.columns)
# decomposed_df.drop('awards', axis=1, inplace=True)
# decomposed_df.drop('awards_description', axis=1, inplace=True)
if not os.path.exists(save_csv_path):
    # Create a new directory because it does not exist
    os.makedirs(save_csv_path)
decomposed_df.to_csv(f"{save_csv_path}\\summary.csv", index=False)

In [60]:
cols

['id',
 'name',
 'Major',
 'Summary',
 'gpa',
 'gpa_description',
 'mastery of language',
 'mastery of language_description',
 'math ability',
 'math ability_description',
 'specialty core courses',
 'specialty core courses_description',
 'lab courses',
 'lab courses_description',
 'capstone/design project',
 'capstone/design project_description',
 'internship',
 'internship_description',
 'community service',
 'community service_description',
 'student clubs',
 'student clubs_description',
 "major int'l/national competition",
 "major int'l/national competition_description",
 'college of bachelor degree',
 'college of bachelor degree_description',
 'college of higher degree',
 'college of higher degree_description',
 'llm pred score',
 'llm pred pass']

In [ ]:
save_csv_path = "SAVE_CSV_PATH"

In [150]:
summary_df = pd.read_csv(f"{save_csv_path}\\summary.csv")
#encoding = "ISO-8859-1" if summary.csv is modified 

In [ ]:
# Comparison of two versions' missing categories
# 
# v5_save_csv_path = "SAVE_CSV_PATH"
# v12_save_csv_path = "SAVE_CSV_PATH"
# v5_summary_df = pd.read_csv(f"{v5_save_csv_path}\\summary.csv")
# v12_summary_df = pd.read_csv(f"{v12_save_csv_path}\\summary.csv",encoding = "ISO-8859-1")
# 
# v5_lab=v5_summary_df.loc[v5_summary_df['community service']==-1][['id','community service_description']]
# v12_lab=v12_summary_df.loc[v12_summary_df['community service']==70][['id','community service','community service_description']]
# 
# diff_lab=v12_lab
# diff_lab=diff_lab[~diff_lab['id'].isin(v5_lab['id'])]
# v5_summary_df[summary_df['id'].isin(diff_lab['id'])][['id','community service','community service_description']]

In [151]:
def mean_ignore_zeros(series):
    non_zero_values = series[(series != 0) & (series != -1)]
    if len(non_zero_values) == 0:
        return np.nan
    return non_zero_values.mean()

# Define a function to calculate the mean while ignoring zeros and -1
def mean_ignore_zeros_neg_ones(series):
    valid_values = series[(series != 0) & (series != -1)]
    if len(valid_values) == 0:
        return np.nan
    return valid_values.mean()

def weight_penalty_zeros_neg_ones(series, penaltyRate):
    valid_values = series[(series != 0) & (series != -1)]
    if len(valid_values) == 0:
        return 0
    return valid_values.mean()-penaltyRate*(len(series)-len(valid_values))

# Function to count zeros and -1 in df_summary
def count_zeros_neg_ones(series):
    return ((series == 0) | (series == -1)).sum()

# Selecting only the numerical columns for mean calculation
numerical_cols = ['overall score', 'score', 'gpa', 'math ability', 'mastery of language', 'specialty core courses', 'lab courses', 'capstone/design project', 'internship', 'community service', 'student clubs', "major int'l/national competition", 'college of bachelor degree']
print(numerical_cols)
summary_df['overall score'] = summary_df[numerical_cols[2:-1]].apply(mean_ignore_zeros_neg_ones, axis=1)

#calculate score with weight
#GPA, math ability, mastery of language, specialty core courses, and lab courses. Weight: 0.125 respectively. 
penaltyRate=3
important_categories_mean=summary_df[numerical_cols[2:7]].apply(weight_penalty_zeros_neg_ones, axis=1, args=(penaltyRate,))
#capstone/design project, Internship, Community service, Student clubs, and Major Int'l/National Competition. Weight: 0.075
unimportant_categories_mean=summary_df[numerical_cols[7:12]].apply(weight_penalty_zeros_neg_ones, axis=1, args=(penaltyRate,))
summary_df['weight score']=important_categories_mean*0.625+unimportant_categories_mean*0.375

# Calculate mean evaluations for each student, excluding non-numerical columns
grouped = df.groupby('id')[numerical_cols[:1]+numerical_cols[2:]].agg(mean_ignore_zeros_neg_ones)

# Rename columns in the grouped dataframe to differentiate from df_summary
grouped.columns = [f'mean_{col}' for col in grouped.columns]
score = df.groupby('id')['score'].mean().reset_index()
# Merge the summary dataframe with the mean evaluations
merged_df = pd.merge(summary_df, grouped, on='id', how='outer')
merged_df = pd.merge(merged_df, score, on='id', how='outer')
merged_df.to_csv(f"{save_csv_path}\\summary_with_prof_eval.csv", index=False)

['overall score', 'score', 'gpa', 'math ability', 'mastery of language', 'specialty core courses', 'lab courses', 'capstone/design project', 'internship', 'community service', 'student clubs', "major int'l/national competition", 'college of bachelor degree']


In [98]:
numerical_cols

['overall score',
 'score',
 'gpa',
 'math ability',
 'mastery of language',
 'specialty core courses',
 'lab courses',
 'capstone/design project',
 'internship',
 'community service',
 'student clubs',
 "major int'l/national competition",
 'college of bachelor degree']

In [ ]:
def count_zeros_neg_ones(series):
    return ((series == 0) | (series == -1)).sum()
abs_errors = pd.DataFrame()

for col in numerical_cols[2:]:
    summary_col = col.lower()
    mean_col = f'mean_{summary_col}'
    valid_rows = (merged_df[summary_col] != -1) & (merged_df[summary_col] != 0)
    abs_errors[summary_col] = (merged_df.loc[valid_rows, summary_col] - merged_df.loc[valid_rows, mean_col]).abs()

# Calculate the MAE for each column
mae = abs_errors.mean()
zero_neg_one_counts = summary_df[numerical_cols[2:-1]].apply(count_zeros_neg_ones)
# Display the MAE
print("Mean Absolute Error (MAE):")
print(mae)
print("\nCount of 0 or -1 in each column of df_summary:")
print(zero_neg_one_counts)